### **GENETIC ALGORITHM**

In [ ]:
import random

POPULATION_SIZE = int(input("Enter Population size: "))
GENE_LENGTH = int(input("Enter Gene length: "))
MUTATION_RATE = float(input("Enter Mutation rate: "))
GENERATIONS = int(input("Enter Generations: "))

def fitness(x):
    return x**2

def decode(binary_str):
    return int(binary_str, 2)

def create_population():
    return [''.join(random.choice('01') for _ in range(GENE_LENGTH)) for _ in range(POPULATION_SIZE)]

def evaluate_population(population):
    return [fitness(decode(individual)) for individual in population]

def select_parents(population, fitnesses):
    total_fitness = sum(fitnesses)
    selection_probs = [f / total_fitness for f in fitnesses]
    return random.choices(population, weights=selection_probs, k=2)

def crossover(parent1, parent2):
    point = random.randint(1, GENE_LENGTH - 1)
    return parent1[:point] + parent2[point:], parent2[:point] + parent1[point:]

def mutate(individual):
    return ''.join(bit if random.random() > MUTATION_RATE else random.choice('01') for bit in individual)

def genetic_algorithm():
    population = create_population()

    for generation in range(GENERATIONS):
        fitnesses = evaluate_population(population)
        new_population = []

        for _ in range(POPULATION_SIZE // 2):
            parent1, parent2 = select_parents(population, fitnesses)
            offspring1, offspring2 = crossover(parent1, parent2)
            new_population.append(mutate(offspring1))
            new_population.append(mutate(offspring2))

        population = new_population

    best_individual = max(population, key=lambda ind: fitness(decode(ind)))
    best_fitness = fitness(decode(best_individual))

    return decode(best_individual), best_fitness

if __name__ == "__main__":
    best_solution, best_fitness = genetic_algorithm()
    print(f"Best solution: {best_solution}, Fitness: {best_fitness}")


Enter Population size: 10
Enter Gene length: 5
Enter Mutation rate: 0.01
Enter Generations: 100
Best solution: 31, Fitness: 961


## **PARTICLE SWARM OPTIMIZATION**

In [ ]:
import numpy as np

def objective_function(x):
    return -x**2 + 5*x + 20

def initialize_particles(n_particles, bounds):
    positions = np.random.uniform(bounds[0], bounds[1], n_particles)
    velocities = np.zeros(n_particles)
    return positions, velocities

def pso(n_particles, bounds, max_iter, w, c1, c2):
    positions, velocities = initialize_particles(n_particles, bounds)
    pbest_positions = np.copy(positions)
    pbest_values = objective_function(pbest_positions)
    gbest_position = pbest_positions[np.argmax(pbest_values)]
    gbest_value = max(pbest_values)

    for iteration in range(max_iter):
        print(f"Iteration {iteration + 1}:")

        for i in range(n_particles):
            r1, r2 = np.random.rand(), np.random.rand()
            velocities[i] = (w * velocities[i] +
                             c1 * r1 * (pbest_positions[i] - positions[i]) +
                             c2 * r2 * (gbest_position - positions[i]))
            positions[i] += velocities[i]

            fitness_value = objective_function(positions[i])
            print(f"Particle {i+1}: Position = {positions[i]:.4f}, Velocity = {velocities[i]:.4f}, Fitness = {fitness_value:.4f}")

            if fitness_value > pbest_values[i]:
                pbest_positions[i] = positions[i]
                pbest_values[i] = fitness_value

        if max(pbest_values) > gbest_value:
            gbest_position = pbest_positions[np.argmax(pbest_values)]
            gbest_value = max(pbest_values)

        print(f"Global Best Position = {gbest_position:.4f}, Global Best Fitness = {gbest_value:.4f}\n")

    return gbest_position, gbest_value

if __name__ == "__main__":
    n_particles = int(input("Enter the number of particles: "))
    bounds = (-10, 10)
    max_iter = int(input("Enter the number of iterations: "))
    w = float(input("Enter the inertia weight (e.g., 0.5): "))
    c1 = float(input("Enter the cognitive constant (e.g., 1.5): "))
    c2 = float(input("Enter the social constant (e.g., 1.5): "))

    best_position, best_value = pso(n_particles, bounds, max_iter, w, c1, c2)
    print(f"Optimal Solution: Position = {best_position:.4f}, Value = {best_value:.4f}")

Enter the number of particles: 10
Enter the number of iterations: 2
Enter the inertia weight (e.g., 0.5): 0.5
Enter the cognitive constant (e.g., 1.5): 1.5
Enter the social constant (e.g., 1.5): 1.5
Iteration 1:
Particle 1: Position = 1.5037, Velocity = 9.2713, Fitness = 25.2573
Particle 2: Position = 1.8011, Velocity = 8.0179, Fitness = 25.7615
Particle 3: Position = -2.2164, Velocity = 4.6087, Fitness = 4.0060
Particle 4: Position = -0.8422, Velocity = 4.4319, Fitness = 15.0794
Particle 5: Position = 2.1134, Velocity = 0.0000, Fitness = 26.1005
Particle 6: Position = 5.6592, Velocity = -0.6365, Fitness = 16.2698
Particle 7: Position = 3.3347, Velocity = -2.9856, Fitness = 25.5533
Particle 8: Position = 6.0675, Velocity = 12.3826, Fitness = 13.5232
Particle 9: Position = 4.2930, Velocity = 12.4235, Fitness = 23.0352
Particle 10: Position = 2.8787, Velocity = -4.0880, Fitness = 26.1066
Global Best Position = 2.8787, Global Best Fitness = 26.1066

Iteration 2:
Particle 1: Position = 7.0

## **ANT COLONY OPTIMIZATION**

In [ ]:
import numpy as np

class AntColony:
    def __init__(self, distance_matrix, num_ants, num_iterations, decay):
        self.distance_matrix = distance_matrix
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.decay = decay
        self.num_cities = len(distance_matrix)
        self.pheromone = np.ones((self.num_cities, self.num_cities))

    def run(self):
        best_path = None
        best_distance = float('inf')

        for _ in range(self.num_iterations):
            all_paths = self.generate_all_paths()
            for path, distance in all_paths:
                if distance < best_distance:
                    best_path = path
                    best_distance = distance
            self.update_pheromones(all_paths)

            self.pheromone *= (1 - self.decay)

        return best_path, best_distance

    def generate_all_paths(self):
        all_paths = []
        for _ in range(self.num_ants):
            path = self.generate_path()
            distance = self.calculate_total_distance(path)
            all_paths.append((path, distance))
        return all_paths

    def generate_path(self):
        path = [0]
        visited = {0}

        while len(path) < self.num_cities:
            current_city = path[-1]
            next_city = self.choose_next_city(current_city, visited)
            path.append(next_city)
            visited.add(next_city)

        path.append(0)
        return path

    def choose_next_city(self, current_city, visited):
        probabilities = []
        for city in range(self.num_cities):
            if city not in visited:
                probabilities.append(self.pheromone[current_city][city] / self.distance_matrix[current_city][city])
            else:
                probabilities.append(0)

        probabilities = np.array(probabilities)
        probabilities /= probabilities.sum() if probabilities.sum() > 0 else 1

        return np.random.choice(range(self.num_cities), p=probabilities)

    def calculate_total_distance(self, path):
        return sum(self.distance_matrix[path[i]][path[i + 1]] for i in range(len(path) - 1))

    def update_pheromones(self, all_paths):
        for path, distance in all_paths:
            for i in range(len(path) - 1):
                self.pheromone[path[i]][path[i + 1]] += 1.0 / distance

if __name__ == "__main__":
    num_cities = int(input("Enter the number of cities: "))
    print("Enter the distance matrix row by row, separated by spaces (use 0 for the diagonal):")

    distance_matrix = []
    for i in range(num_cities):
        row = list(map(float, input(f"Row {i + 1}: ").strip().split()))
        if len(row) != num_cities:
            print("Error: Each row must have the same number of elements as the number of cities.")
            exit(1)
        distance_matrix.append(row)

    distance_matrix = np.array(distance_matrix)

    num_ants = int(input("Enter the number of ants: "))
    num_iterations = int(input("Enter the number of iterations: "))
    decay = float(input("Enter the pheromone decay rate (e.g., 0.1): "))

    aco = AntColony(distance_matrix, num_ants, num_iterations, decay)

    best_path, best_distance = aco.run()

    print("Best path: ", best_path)
    print("Best distance: ", best_distance)

Enter the number of cities: 4
Enter the distance matrix row by row, separated by spaces (use 0 for the diagonal):
Row 1: 0 4 15 1
Row 2: 4 0 5 8
Row 3: 15 5 0 4
Row 4: 1 8 4 0
Enter the number of ants: 2
Enter the number of iterations: 3
Enter the pheromone decay rate (e.g., 0.1): 0.1
Best path:  [0, 3, 2, 1, 0]
Best distance:  14.0


## **CUCKOO SEARCH ALGORITHM**

In [1]:
import numpy as np
import math

def objective_function(x):
    return np.sum(x**2)

def levy_flight(Lambda):
    sigma = (math.gamma(1 + Lambda) * np.sin(np.pi * Lambda / 2) /
             (math.gamma((1 + Lambda) / 2) * Lambda * 2**((Lambda - 1) / 2)))**(1 / Lambda)
    u = np.random.normal(0, sigma, size=dimension)
    v = np.random.normal(0, 1, size=dimension)
    step = u / np.abs(v)**(1 / Lambda)
    return step

def cuckoo_search(n=5, max_iter=5, lb=-5, ub=5, pa=0.25, Lambda=1.5):
    nests = np.random.uniform(lb, ub, (n, dimension))
    fitness = np.array([objective_function(nest) for nest in nests])
    best_nest = nests[np.argmin(fitness)]
    best_fitness = np.min(fitness)

    for iteration in range(max_iter):
        for i in range(n):
            step_size = levy_flight(Lambda)
            new_nest = nests[i] + step_size
            new_nest = np.clip(new_nest, lb, ub)
            new_fitness = objective_function(new_nest)

            if new_fitness < fitness[i]:
                nests[i] = new_nest
                fitness[i] = new_fitness

        for i in range(n):
            if np.random.rand() < pa:
                nests[i] = np.random.uniform(lb, ub, dimension)
                fitness[i] = objective_function(nests[i])

        current_best_fitness = np.min(fitness)
        if current_best_fitness < best_fitness:
            best_fitness = current_best_fitness
            best_nest = nests[np.argmin(fitness)]

        print(f"Iteration {iteration + 1}/{max_iter}, Best Fitness: {best_fitness}")

    print("\nBest Solution Found:")
    print(f"Solution: {best_nest}")
    print(f"Fitness: {best_fitness}")

dimension = int(input("Number of dimensions (variables): "))
nests = int(input("Number of nests (population size): "))
max_iterations = int(input("Maximum number of iterations: "))
lb = int(input("Lower bound for the solution space: "))
ub = int(input("Upper bound for the solution space: "))
pa = float(input("Probability of discovering an egg: "))

cuckoo_search(n=nests, max_iter=max_iterations, lb=lb, ub=ub, pa=pa)

Number of dimensions (variables): 5
Number of nests (population size): 5
Maximum number of iterations: 5
Lower bound for the solution space: -5
Upper bound for the solution space: 5
Probability of discovering an egg: .25
Iteration 1/5, Best Fitness: 14.086236824663839
Iteration 2/5, Best Fitness: 14.086236824663839
Iteration 3/5, Best Fitness: 14.086236824663839
Iteration 4/5, Best Fitness: 14.086236824663839
Iteration 5/5, Best Fitness: 14.086236824663839

Best Solution Found:
Solution: [-0.37819658  2.55772556  1.41438381  3.33905656 -4.84023401]
Fitness: 14.086236824663839


## **GREY WOLF OPTIMIZATION**

In [ ]:
import numpy as np
def rastrigin_function(x):
    A = 10
    return A * len(x) + np.sum(x**2 - A * np.cos(2 * np.pi * x))
class GreyWolfOptimizer:
    def __init__(self, cost_function, dim, pop_size, max_iter, lb, ub):
        self.cost_function = cost_function
        self.dim = dim
        self.pop_size = pop_size
        self.max_iter = max_iter
        self.lb = lb
        self.ub = ub
        self.alpha_position = np.zeros(dim)
        self.beta_position = np.zeros(dim)
        self.delta_position = np.zeros(dim)
        self.alpha_score = float('inf')
        self.beta_score = float('inf')
        self.delta_score = float('inf')
        self.positions = np.random.uniform(lb, ub, (pop_size, dim))
    def optimize(self):
        for t in range(self.max_iter):
            for i in range(self.pop_size):
                fitness = self.cost_function(self.positions[i])
                if fitness < self.alpha_score:
                    self.alpha_score = fitness
                    self.alpha_position = self.positions[i]
                elif fitness < self.beta_score:
                    self.beta_score = fitness
                    self.beta_position = self.positions[i]
                elif fitness < self.delta_score:
                    self.delta_score = fitness
                    self.delta_position = self.positions[i]
            A1 = 2 * np.random.random(self.dim) - 1
            A2 = 2 * np.random.random(self.dim) - 1
            A3 = 2 * np.random.random(self.dim) - 1
            C1 = 2 * np.random.random(self.dim)
            C2 = 2 * np.random.random(self.dim)
            C3 = 2 * np.random.random(self.dim)
            for i in range(self.pop_size):
                D_alpha = np.abs(C1 * self.alpha_position - self.positions[i])
                D_beta = np.abs(C2 * self.beta_position - self.positions[i])
                D_delta = np.abs(C3 * self.delta_position - self.positions[i])
                self.positions[i] = self.positions[i] + A1 * D_alpha + A2 * D_beta + A3 * D_delta
                self.positions[i] = np.clip(self.positions[i], self.lb, self.ub)
            if (t + 1) % 10 == 0:
                print(f"Iter = {t+1} best fitness = {self.alpha_score:.3f}")
        return self.alpha_position, self.alpha_score
if __name__ == "__main__":
    dim = 3
    pop_size = 50
    max_iter = 100
    lb = -5.12
    ub = 5.12
    gwo = GreyWolfOptimizer(cost_function=rastrigin_function, dim=dim, pop_size=pop_size, max_iter=max_iter, lb=lb, ub=ub)
    best_position, best_score = gwo.optimize()
    print("\nGWO completed")
    print(f"\nBest solution found: {best_position}")
    print(f"Fitness of best solution = {best_score:.6f}")
    print("\nEnd GWO for Rastrigin function")

Iter = 10 best fitness = 6.250
Iter = 20 best fitness = 4.284
Iter = 30 best fitness = 4.284
Iter = 40 best fitness = 4.284
Iter = 50 best fitness = 4.284
Iter = 60 best fitness = 4.284
Iter = 70 best fitness = 4.284
Iter = 80 best fitness = 4.284
Iter = 90 best fitness = 4.284
Iter = 100 best fitness = 4.284

GWO completed

Best solution found: [0.46321201 1.17742145 4.11338956]
Fitness of best solution = 4.283780

End GWO for Rastrigin function


##**PARALLEL CELLULAR ALGORITHM**

In [2]:
import numpy as np
from multiprocessing import Pool

def fitness_function(x):
    return x**2 - 4*x + 4

def update_cell(cell_info):
    x, neighbors = cell_info
    new_value = np.mean(neighbors)
    return new_value

def parallel_cellular_algorithm(grid_size, num_iterations):
    grid = np.random.uniform(-10, 10, (grid_size, grid_size))
    best_solution = None
    best_fitness = float('inf')

    def get_neighbors(i, j):
        neighbors = []
        for di in [-1, 0, 1]:
            for dj in [-1, 0, 1]:
                if di == 0 and dj == 0:
                    continue
                ni, nj = (i + di) % grid_size, (j + dj) % grid_size
                neighbors.append(grid[ni, nj])
        return neighbors

    for iteration in range(num_iterations):
        inputs = []
        for i in range(grid_size):
            for j in range(grid_size):
                neighbors = get_neighbors(i, j)
                inputs.append((grid[i, j], neighbors))

        with Pool() as pool:
            updated_values = pool.map(update_cell, inputs)

        updated_grid = np.array(updated_values).reshape(grid_size, grid_size)
        grid = updated_grid

        for i in range(grid_size):
            for j in range(grid_size):
                fitness = fitness_function(grid[i, j])
                if fitness < best_fitness:
                    best_fitness = fitness
                    best_solution = grid[i, j]

        print(f"Iteration {iteration + 1}/{num_iterations}:")
        print(f"  Current Best Solution: {best_solution}")
        print(f"  Current Best Fitness: {best_fitness:.4f}")
        print(f"  Average Fitness of Grid: {np.mean([fitness_function(grid[i, j]) for i in range(grid_size) for j in range(grid_size)]):.4f}")
        print("-" * 50)

    return best_solution, best_fitness

if __name__ == "__main__":
    grid_size = int(input("Enter grid size: "))
    num_iterations = int(input("Enter number of iterations: "))

    print("\nStarting Parallel Cellular Algorithm...")
    print(f"Grid Size: {grid_size}x{grid_size}")
    print(f"Number of Iterations: {num_iterations}")
    print("-" * 50)

    solution, fitness = parallel_cellular_algorithm(grid_size, num_iterations)

    print("\nAlgorithm Completed!")
    print("=" * 50)
    print(f"Best Solution Found: {solution}")
    print(f"Fitness of Best Solution: {fitness:.4f}")
    print("=" * 50)

Enter grid size: 5
Enter number of iterations: 3

Starting Parallel Cellular Algorithm...
Grid Size: 5x5
Number of Iterations: 3
--------------------------------------------------
Iteration 1/3:
  Current Best Solution: 2.2200113012706075
  Current Best Fitness: 0.0484
  Average Fitness of Grid: 2.3029
--------------------------------------------------
Iteration 2/3:
  Current Best Solution: 2.017781981885933
  Current Best Fitness: 0.0003
  Average Fitness of Grid: 0.7367
--------------------------------------------------
Iteration 3/3:
  Current Best Solution: 2.017781981885933
  Current Best Fitness: 0.0003
  Average Fitness of Grid: 0.5773
--------------------------------------------------

Algorithm Completed!
Best Solution Found: 2.017781981885933
Fitness of Best Solution: 0.0003


##**OPTIMIZATION VIA GENE EXPRESSION**

In [3]:
import random

def fitness_function(x):
    return -1 * (x**2 - 4*x + 4)

def decode(chromosome, lower_bound, upper_bound):
    max_value = 2**len(chromosome) - 1
    decoded = int(chromosome, 2)
    return lower_bound + (decoded / max_value) * (upper_bound - lower_bound)

def initialize_population(population_size, gene_length):
    return [''.join(random.choice('01') for _ in range(gene_length)) for _ in range(population_size)]

def evaluate_population(population, lower_bound, upper_bound):
    return [fitness_function(decode(ind, lower_bound, upper_bound)) for ind in population]

def select_parents(population, fitnesses):
    total_fitness = sum(fitnesses)
    selection_probs = [f / total_fitness for f in fitnesses]
    return random.choices(population, weights=selection_probs, k=2)

def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 1)
    return parent1[:point] + parent2[point:], parent2[:point] + parent1[point:]

def mutate(chromosome, mutation_rate):
    return ''.join(bit if random.random() > mutation_rate else random.choice('01') for bit in chromosome)

def gene_expression_algorithm(population_size, gene_length, mutation_rate, crossover_rate, generations, lower_bound, upper_bound):
    population = initialize_population(population_size, gene_length)
    best_solution = None
    best_fitness = float('-inf')

    for generation in range(generations):
        fitnesses = evaluate_population(population, lower_bound, upper_bound)

        for ind, fit in zip(population, fitnesses):
            if fit > best_fitness:
                best_fitness = fit
                best_solution = ind

        print(f"Generation {generation + 1}/{generations}")
        print(f"  Best Solution So Far: {decode(best_solution, lower_bound, upper_bound):.4f}")
        print(f"  Best Fitness So Far: {best_fitness:.4f}")
        print(f"  Average Fitness: {sum(fitnesses) / len(fitnesses):.4f}")
        print("-" * 50)

        new_population = []
        for _ in range(population_size // 2):
            parent1, parent2 = select_parents(population, fitnesses)
            if random.random() < crossover_rate:
                offspring1, offspring2 = crossover(parent1, parent2)
            else:
                offspring1, offspring2 = parent1, parent2
            new_population.append(mutate(offspring1, mutation_rate))
            new_population.append(mutate(offspring2, mutation_rate))

        population = new_population

    decoded_best = decode(best_solution, lower_bound, upper_bound)
    return decoded_best, best_fitness

if __name__ == "__main__":
    print("Gene Expression Algorithm for Optimization\n")
    population_size = int(input("Enter population size: "))
    gene_length = int(input("Enter gene length: "))
    mutation_rate = float(input("Enter mutation rate (e.g., 0.01): "))
    crossover_rate = float(input("Enter crossover rate (e.g., 0.7): "))
    generations = int(input("Enter number of generations: "))
    lower_bound = float(input("Enter lower bound of the solution space: "))
    upper_bound = float(input("Enter upper bound of the solution space: "))

    print("\nStarting Gene Expression Algorithm...")
    print("-" * 50)

    best_solution, best_fitness = gene_expression_algorithm(
        population_size, gene_length, mutation_rate, crossover_rate, generations, lower_bound, upper_bound
    )

    print("\nAlgorithm Completed!")
    print("=" * 50)
    print(f"Best Solution Found: {best_solution:.4f}")
    print(f"Fitness of Best Solution: {best_fitness:.4f}")
    print("=" * 50)


Gene Expression Algorithm for Optimization

Enter population size: 5
Enter gene length: 3
Enter mutation rate (e.g., 0.01): 0.1
Enter crossover rate (e.g., 0.7): 0.8
Enter number of generations: 3
Enter lower bound of the solution space: -10
Enter upper bound of the solution space: 10

Starting Gene Expression Algorithm...
--------------------------------------------------
Generation 1/3
  Best Solution So Far: -4.2857
  Best Fitness So Far: -39.5102
  Average Fitness: -58.1224
--------------------------------------------------
Generation 2/3
  Best Solution So Far: -4.2857
  Best Fitness So Far: -39.5102
  Average Fitness: -45.6327
--------------------------------------------------
Generation 3/3
  Best Solution So Far: -1.4286
  Best Fitness So Far: -11.7551
  Average Fitness: -38.6939
--------------------------------------------------

Algorithm Completed!
Best Solution Found: -1.4286
Fitness of Best Solution: -11.7551
